In [ ]:
from bucket_manager import bucket_manager as bm
import swiftclient
import os
from subprocess import Popen, PIPE

In [ ]:
from dask import dataframe as dd
import pandas as pd

In [ ]:
if 'ST_AUTH' not in os.environ:
    os.environ['ST_AUTH'] = 'https://s3.echo.stfc.ac.uk/auth/1.0'
if 'ST_USER' not in os.environ:
    os.environ['ST_USER'] = Popen(['grep', 'user', os.path.expanduser('~/.keys/lsst-swift-credentials.json')], stdout=PIPE).stdout.read().decode('utf-8').split('"')[3].strip()
if 'ST_KEY' not in os.environ:
    os.environ['ST_KEY'] = Popen(['grep', 'secret_key', os.path.expanduser('~/.keys/lsst-swift-credentials.json')], stdout=PIPE).stdout.read().decode('utf-8').split('"')[3].strip()

In [ ]:
bm.check_keys()

In [ ]:
conn = bm.get_conn_swift()

In [ ]:
from_container = 'LSST-IR-FUSION-Butlers'
to_container = 'LSST-IR-FUSION-Butlers-wide'

In [ ]:
bm.bucket_list_swift(conn)

In [ ]:
assert from_container in bm.bucket_list_swift(conn), f"Source container {from_container} does not exist"
assert to_container not in bm.bucket_list_swift(conn), f"Destination container {to_container} already exists"

In [ ]:
bm.create_bucket_swift(conn, to_container)

In [ ]:
prefix = 'butler_wide_20220930'

In [ ]:
from_objects = bm.object_list_swift(conn, from_container, prefix='butler_wide_20220930', full_listing=True)

In [ ]:
len(from_objects)

In [ ]:
def copy_object(row, conn, from_container, to_container):
    object_name = row['object']
    try:
        conn.copy_object(from_container, object_name, destination=f'/{to_container}/{object_name}')
    except swiftclient.ClientException as e:
        print(f"Error copying {object_name}: {e}")
        return False
    print(f"Copied {object_name} from {from_container} to {to_container}")
    return True

In [ ]:
ddf = dd.from_pandas(pd.DataFrame(from_objects, columns=['object']), npartitions=100)

In [ ]:
ddf

In [ ]:
ddf['copied'] = ddf.map_partitions(
    lambda partition: partition.apply(
        copy_object,
        axis=1,
        args=(conn, from_container, to_container),
    ),
    meta=('copied', 'bool')
)

In [ ]:
ddf

In [ ]:
def delete_original(row, conn, container):
    object_name = row['object']
    if not row['copied']:
        print(f"Skipping deletion of {object_name} as copy failed")
        return False
    try:
        conn.delete_object(container, object_name)
    except swiftclient.ClientException as e:
        print(f"Error deleting {object_name}: {e}")
        return False
    print(f"Deleted {object_name} from {container}")
    return True

In [ ]:
# if all copied, delete the original object
ddf['deleted_original'] = ddf.map_partitions(
    lambda partition: partition.apply(
        delete_original,
        axis=1,
        args=(conn, from_container)
    ),
    meta=('deleted_original', 'bool')
)

In [ ]:
ddf.to_csv('copy_results.csv', single_file=True, index=False)